# 🛠️ Titanic Survival Prediction - Baseline Model

This notebook is part of the Titanic Survival Prediction project. In this phase, we set up a mlflow server on dagshub and build a baseline model.

[MLflow Server](https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow)

In [1]:
!pip install mlflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninst

In [2]:
!pip install dagshub==0.3.34

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.6/236.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.5 MB/s eta 0:00:00
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10486 sha256=37c0cddd0c2fd6c4436f9e3172d72d31b2bd398fcc0b3e841c9ad7a2160cfa21
  Stored in directory: /root/.cache/pip/wheels/db/4a/86/fdda91f8b8ebb0a70e4181dc2423b1f70c3c2d3bd1158685b5
Successfully built fusepy
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: rich
    Found exist

In [3]:
import mlflow
import mlflow.sklearn
import dagshub
import os

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
dagshub_token = user_secrets.get_secret("DAGSHUB_PAT")

In [5]:
if not dagshub_token:
    raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

In [6]:
dagshub_url = "https://dagshub.com"
repo_owner = "pxxthik"
repo_name = "Titanic-Survival-Prediction"

# Set up MLflow tracking URI
mlflow.set_tracking_uri(f'{dagshub_url}/{repo_owner}/{repo_name}.mlflow')

In [7]:
with mlflow.start_run():
    mlflow.log_param("Parameter", "value")
    mlflow.log_metric("Metric", 1)

2025/06/18 16:52:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run burly-kite-413 at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/0/runs/163ed6d380394320ab2563ff58456d0a.
2025/06/18 16:52:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/0.


In [8]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [9]:
df = pd.read_csv("/kaggle/input/titanic-features/titanic_features.csv")
X = df.drop(columns=["Survived"])
y = df["Survived"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=42)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 10), (179, 10), (712,), (179,))

In [12]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
y_pred = model.predict(X_test)

In [14]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"ROC AUC: {roc_auc}")

Accuracy: 0.7932960893854749
Precision: 0.7761194029850746
Recall: 0.7027027027027027
ROC AUC: 0.7799227799227799


In [15]:
import json

metrics = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "roc_auc": roc_auc,
}

with open("metrics.json", 'w') as file:
    json.dump(metrics, file, indent=4)

In [16]:
mlflow.set_experiment("baseline model")
with mlflow.start_run():
    
    if hasattr(model, 'get_params'):
        params = model.get_params()
        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)

    for key, value in metrics.items():
        mlflow.log_metric(key, value)

    mlflow.log_artifact('/kaggle/working/metrics.json')
    mlflow.sklearn.log_model(model, "baseline_model")

2025/06/18 16:52:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpvxkerl8f/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.2.2', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/06/18 16:52:41 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/06/18 16:52:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run popular-rat-751 at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/1/runs/8a6bb03d5d8140c5b4c4d89b53707728.
2025/06/18 16:52:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/1.


In [17]:
mlflow.__version__

'2.15.0'

In [18]:
dagshub.__version__

'0.3.34'

In [19]:
!pip uninstall dagshub -y

Found existing installation: dagshub 0.3.34
Uninstalling dagshub-0.3.34:
  Successfully uninstalled dagshub-0.3.34


In [20]:
import dagshub